In [ ]:
import dynamo as dyn
dyn.get_all_dependencies_version()

import scanpy as sc
import anndata
from scipy import io
from scipy.sparse import coo_matrix, csr_matrix
import numpy as np  #
import os
import pandas as pd  ###
import matplotlib.pyplot as plt

import scvelo as scv

from gseapy.plot import barplot, dotplot

%matplotlib inline
from dynamo.dynamo_logger import main_info, LoggerManager
LoggerManager.main_logger.setLevel(LoggerManager.INFO)


import seaborn as sns

import sys




In [ ]:
# visualization settings
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

plt.rcParams['figure.figsize'] = [6, 4.5]
plt.rcParams["savefig.dpi"] = 1000

save_folder = "figure"
os.makedirs(save_folder, exist_ok=True)

In [ ]:
dyn.configuration.set_figure_params('dynamo', background='white') # jupter notebooks
dyn.configuration.set_pub_style() # manuscript

In [ ]:
adata = sc.read_h5ad('cornea_epithelium_final_dynamo.h5ad')

In [ ]:
adata.obs.leiden_anno.unique()

In [ ]:
dyn.pp.recipe_monocle(adata)
dyn.tl.dynamics(adata, cores=48)
dyn.tl.reduceDimension(adata)

dyn.pl.umap(adata, color='leiden_anno')

In [ ]:
# you can set `verbose = 1/2/3` to obtain different levels of running information of vector field reconstruction
dyn.vf.VectorField(adata, basis='umap', M=1000, pot_curl_div=True)

In [ ]:
dyn.pl.umap(adata,  color=['umap_ddhodge_potential',"RORA","PITX1"],layer="M_s", frontier=True)

#plt.savefig("./figure/cornea umap_ddhodge_potential.pdf")

In [ ]:
dyn.tl.cell_velocities(adata, basis='pca')
dyn.vf.VectorField(adata, basis='pca')
dyn.vf.speed(adata, basis='pca')
dyn.vf.curl(adata, basis='umap')
dyn.vf.divergence(adata, basis='pca')
dyn.vf.acceleration(adata, basis='pca')
dyn.vf.curvature(adata, basis='pca')

In [ ]:
fig1, f1_axes = plt.subplots(ncols=2, nrows=2, constrained_layout=True, figsize=(12, 8))
f1_axes
f1_axes[0, 0] = dyn.pl.cell_wise_vectors(adata, color='speed_pca', pointsize=0.5, alpha = 0.7, ax=f1_axes[0, 0], quiver_length=6, quiver_size=6, save_show_or_return='return')
f1_axes[0, 1] = dyn.pl.grid_vectors(adata, color='divergence_pca', ax=f1_axes[0, 1], quiver_length=12, quiver_size=12, save_show_or_return='return')
f1_axes[1, 0] = dyn.pl.streamline_plot(adata, color='acceleration_pca', ax=f1_axes[1, 0], save_show_or_return='return')
f1_axes[1, 1] = dyn.pl.streamline_plot(adata, color='curvature_pca', ax=f1_axes[1, 1], save_show_or_return='return')
plt.show()


plt.savefig("./figure/cornea speed and acceleration divergence curvature pca.pdf")


In [ ]:
dyn.tl.cell_velocities(adata,basis='umap')
dyn.vf.VectorField(adata,basis='umap', M=1000,cores=48)

In [ ]:
dyn.pl.streamline_plot(
    adata,
    color=["leiden_anno"],
    layer="M_s",
    figsize=(8, 6),
    #ncols=2
)

#plt.savefig("./figure/streamline_plot_M_s.svg")

In [ ]:
dyn.pp.top_pca_genes(adata, n_top_genes=500);  

In [ ]:
top_pca_genes = adata.var.index[adata.var.top_pca_genes];

top_pca_genes


In [ ]:
dyn.vf.jacobian(adata, regulators=top_pca_genes, effectors=top_pca_genes);

In [ ]:
dyn.pl.jacobian(adata, regulators=['RORA'], effectors=['RORA','PITX1'], basis='umap', 
                figsize=(6, 4), show_arrowed_spines=True, stacked_fraction=False)


In [ ]:
#%matplotlib inline
 

dyn.vf.jacobian(adata, regulators=["RORA"], effectors=['PITX1','KRT3',"IFITM3","KRT14","KRT19", "KLF4","TGFBI",
                                                                "MAL","KRT12","BCAM","LGALS3","NQO1","S100A2","KRT15"] )
dyn.pl.response(
    adata,
    np.array([ ["RORA", "PITX1"], ["RORA", "KRT3"], ["RORA", "IFITM3"],["RORA", "KRT14"],["RORA", "KRT19"],["RORA", "KLF4"],
              ["RORA", "TGFBI"],["RORA", "MAL"],
              
              ["RORA", "KRT12"],["RORA", "BCAM"],["RORA", "LGALS3"],["RORA", "NQO1"],["RORA", "S100A2"],["RORA", "KRT15"]
             ]),
    ykey="jacobian",
    xkey="M_s",
     n_col=2, n_row=7,zero_indicator=True,fit_curve=False, fit_mode='hill',
     stacked_fraction=False,
    log=False,
    drop_zero_cells=True,
    grid_num=30,
    figsize=(5, 5),
    save_show_or_return='both',
    save_kwargs={"path":"jacobian","prefix":"RORA","ext":"pdf"}
    
)

In [ ]:

dyn.vf.jacobian(adata, regulators=["PITX1"], effectors=['KRT3',"KRT12"] )
dyn.pl.response(
    adata,
    np.array([ ["PITX1", "KRT3"], ["PITX1", "KRT12"]

             ]),
    
    ykey="jacobian",
    xkey="M_s",
     n_col=2, n_row=1, zero_indicator=True,fit_curve=False, fit_mode='hill',
     stacked_fraction=False,
    log=False,
    drop_zero_cells=True,
    grid_num=30,
    figsize=(5, 5),
    save_show_or_return='both',
    save_kwargs={"path":"jacobian_1","prefix":"PITX1","ext":"pdf"}
    
)




dyn.vf.jacobian(adata, regulators=["PITX1"], effectors=["MAL","CLU"] )
                                                        
dyn.pl.response(
    adata,
    np.array([ ["PITX1", "MAL"],["PITX1", "CLU"],

             ]),
    
    ykey="jacobian",
    xkey="M_s",
     n_col=2, n_row=1, zero_indicator=True,fit_curve=False, fit_mode='hill',
     stacked_fraction=False,
    log=False,
    drop_zero_cells=True,
    grid_num=30,
    figsize=(5, 5),
    save_show_or_return='both',
    save_kwargs={"path":"jacobian_2","prefix":"PITX1","ext":"pdf"}
    
)



dyn.vf.jacobian(adata, regulators=["PITX1"], effectors=["KRT14","KRT19",] )
dyn.pl.response(
    adata,
    np.array([ ["PITX1", "KRT14"], ["PITX1", "KRT19"],

             ]),
    
    ykey="jacobian",
    xkey="M_s",
     n_col=2, n_row=1, zero_indicator=True,fit_curve=False, fit_mode='hill',
     stacked_fraction=False,
    log=False,
    drop_zero_cells=True,
    grid_num=30,
    figsize=(5, 5),
    save_show_or_return='both',
    save_kwargs={"path":"jacobian_3","prefix":"PITX1","ext":"pdf"}
    
)



dyn.vf.jacobian(adata, regulators=["PITX1"], effectors=["S100A2","KRT15"] )
                                                         
dyn.pl.response(
    adata,
    np.array([ ["PITX1", "S100A2"],["PITX1", "KRT15"],

             ]),
    
    ykey="jacobian",
    xkey="M_s",
     n_col=2, n_row=1, zero_indicator=True,fit_curve=False, fit_mode='hill',
     stacked_fraction=False,
    log=False,
    drop_zero_cells=True,
    grid_num=30,
    figsize=(5, 5),
    save_show_or_return='both',
    save_kwargs={"path":"jacobian_4","prefix":"PITX1","ext":"pdf"}
    
)

In [ ]:
dyn.ext.ddhodge(adata, basis='pca')


In [ ]:
#dyn.vf.VectorField(adata, basis='umap')
dyn.ext.ddhodge(adata, basis='umap')

In [ ]:
dyn.pl.streamline_plot(adata, color=[ 'leiden_anno','umap_ddhodge_potential',"RORA","PITX1"],#'pca_ddhodge_potential',
                       ncols=2, basis='umap',layer="M_s",) 

plt.savefig("./figure/cornea RORA PITX1 ddhodge_potential.pdf")


In [ ]:
transition_genes = adata.var_names[adata.var.use_for_transition]
transition_genes

In [ ]:
dyn.pl.kinetic_heatmap(adata,
                       genes=transition_genes,
                       tkey='umap_ddhodge_potential',
                       gene_order_method='maximum',
                       layer='acceleration',
                       mode='pseudotime',
                       angle = 0,
                       #show_colorbar=True,
                       #yticklabels=True,
                       show_colorbar=False,
                       yticklabels=True,
                       figsize=(6, 200),
                       #figsize=(6, 250),
                       color_map='RdBu_r',)
                     #xticklabels=True )
                       

plt.savefig("./figure/umap acceleration heatmap_tmp.pdf")

In [ ]:
selected_genes =  ["RORA", "PITX1"]
expr_vals = [-100, -100]
dyn.pd.perturbation(adata, selected_genes, expr_vals, emb_basis="umap",pertubation_method="j_x_prime")

dyn.pl.streamline_plot(adata, color=["leiden_anno","umap_ddhodge_potential"], basis="umap_perturbation",
                       save_show_or_return='save',save_kwargs={"path":"KO","prefix":"RORA_PITX1","ext":"svg"})




In [ ]:
selected_genes =  ["RORA", "PITX1"]
expr_vals = [100, 100]
dyn.pd.perturbation(adata, selected_genes, expr_vals, emb_basis="umap",pertubation_method="j_delta_x")

dyn.pl.streamline_plot(adata, color=["leiden_anno","umap_ddhodge_potential"], basis="umap_perturbation",
                       save_show_or_return='save',save_kwargs={"path":"OE","prefix":"RORA_PITX1","ext":"svg"})

